In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

In [3]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
url_hemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
# Optional delay for loading the page: wait a second before searching for components since the website may have heavy-loading images
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
# set up the HTML parser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
# pinpoints the <ul /> tag with a class of “item_list”, and the <li /> tag with the class of “slide.”
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
# Get the title: div tag, class = content_title
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8657/nasas-perseverance-rover-mission-getting-in-shape-for-launch/" target="_self">NASA's Perseverance Rover Mission Getting in Shape for Launch</a></div>

In [6]:
# Use the parent element to find the FIRST `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Mission Getting in Shape for Launch"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. '

### Featured Images

In [8]:
# Steps to get the full-size original image size:
# 1. visit the webpage, click the 'Full Image'
# 2. Click "more info" button
# 3. click on the image

In [9]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [11]:
# Find the 'more info' button and click that
# after waiting time, search for the text "more info", make sure we have it first:
browser.is_element_present_by_text('more info', wait_time=1)
# find the link associated with the text:
more_info_elem = browser.find_link_by_partial_text('more info')
# click the link:
more_info_elem.click()

C:\Users\LocNguyen\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [12]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [13]:
# Find the relative image url
#  use all 3 tags: <figure /> (class lede) --> <a /> --> <img />
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA13911_hires.jpg'

In [14]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA13911_hires.jpg'

### Collect table data: Mars Fact

In [15]:
# specifying an index of 0, we’re telling Pandas to pull only the first table it encounters
df = pd.read_html('http://space-facts.com/mars/')[0]

df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

### CHALLENGE:

In [17]:
# Visit the mars nasa news site
url_hemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_hemi)
# Optional delay for loading the page: wait a second before searching for components since the website may have heavy-loading images
browser.is_element_present_by_css("div.collapsible.results", wait_time=1)

True

In [18]:
# set up the HTML parser
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')
# pinpoints the <div /> tag with a class of “collapsible results”
elem = img_soup.select_one('div.collapsible.results')

In [19]:
# Hemisphere list:
hemi_list = []

In [20]:
# Full loop to get all four hemisphere names:
all_hemis = elem.find_all('div', class_='description')
all_img_links = [hemi.select_one('a.itemLink.product-item').get("href") \
                 for hemi in all_hemis]
all_img_links

['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [21]:
# Full loop to get all four img links:
for i in all_img_links:
    base_url = 'https://astrogeology.usgs.gov'
    link = f'{base_url}{i}'
    
    # visit the page:
    browser.visit(link)
    browser.is_element_not_present_by_css("div.wide-image-wrapper", wait_time=1)
        
    # Parse the resulting html with soup
    hemi_html = browser.html
    hemi_html_soup = BeautifulSoup(hemi_html, 'html.parser')
    
    # Find the relative image url
    # use 2 tags: <div /> class=wide-image-wrapper --> <img /> class=wide-image
    img_url_rel = hemi_html_soup.select_one('div.wide-image-wrapper img.wide-image').get("src")
    # Use the base URL to create an absolute URL
    img_url = f'{base_url}{img_url_rel}'
    
    # Find the image title
    # use all 2 tags: <div /> class=content --> <h2 /> class=title
    img_title = hemi_html_soup.select_one('div.content h2.title').text
    
    # Return the result into a dict, then append to the list:
    hemi_dict = {
        "img_url": img_url,
        "title": img_title
    }
    hemi_list.append(hemi_dict)
    
    print(hemi_dict)

{'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}
{'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}
{'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}
{'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}


In [22]:
hemi_list

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [23]:
# Quit the browser to stop using our resources:
browser.quit()